In [213]:
def prepro(data_wu):#preprocessing the dataframe
    data_wu.iloc[:,0]=data_wu.iloc[:,0].apply(lambda x: 'ham' if x==0 else 'spam')
    data_wu=data_wu.rename(columns={'True':'Label'})
    data_wu=pd.concat([data_wu.iloc[:,1:],data_wu.iloc[:,0]],axis=1)
    return data_wu
def stop_or_not(data):# check if the data contains only one classification
    classification,count=np.unique(data['Label'],return_counts=True)
    if len(classification)==1:
        return True
    else:
        return False
def potential_split(data):#get all potential threshold for spliting in to a dictionary
    potential_threshold={}
    for ch in data.columns[:-1]:
        l=np.unique(data[ch])
        nl=[(l[i]+l[i-1])/2 for i in range(1,len(l))]
        potential_threshold[ch]=nl
    return potential_threshold
def split_data(data,col_name,va):   
    return data.copy()[data[col_name]>=va],data.copy()[data[col_name]<va]
def classification(data):#get the classification based on the majority
    temp=np.unique(data['Label'],return_counts=True)
    classname=temp[0][temp[1].argmax()]
    return classname
def optimal_split(data):#get the optimal split column and optimal threshold based on entropy
    global best_col, best_thres
    split_threshold_dict=potential_split(data)
    min_en=100000
    for ch in data.columns[:-1]:
        for t in split_threshold_dict[ch]:
            data1,data2=split_data(data,ch,t)
            p1=len(data1)/len(data)
            p2=len(data2)/len(data)
            tol_en=entropy(data1)*p1+entropy(data2)*p2
            if tol_en<=min_en:
                min_en=tol_en
                best_col=ch
                best_thres=t
    return best_col,best_thres
def entropy(data):#entropy calcualtion
    label,count=np.unique(data['Label'],return_counts=True)
    prob=count/len(data)
    ent=sum(prob*np.log(1/prob))
    return ent
def decision_tree(data,maximum_depth): #the main function of decision tree, generate a trained decision tree dictionary
    if stop_or_not(data):
        return classification(data)
    if maximum_depth==0:
        return classification(data)
    else:
        best_col,best_thres=optimal_split(data)
        data1,data2=split_data(data,best_col,best_thres)        
        d1=decision_tree(data1,maximum_depth-1)
        d2=decision_tree(data2,maximum_depth-1)
        criteria = "{} >= {}".format(best_col, best_thres)
        subtree = {criteria: []}
        subtree[criteria].append(d1)
        subtree[criteria].append(d2)
        return subtree
def test_class(decision,sample):#recursively get the classification just like main function
    col,val=list(decision.keys())[0].split()[0],list(decision.keys())[0].split()[2]
    if sample[col]>=float(val):
        answer=decision[list(decision.keys())[0]][0]
    else:
        answer=decision[list(decision.keys())[0]][1]
    result=['ham','spam']
    if answer in result:
        return answer
    else:
        return test_class(answer,sample)
def accuracy(test,decision):#calculate the accuracy of the trained decision tree
    tol_res=[]
    y_true=test['Label']
    for i in range(len(test)):
        temp=test.iloc[i,:]
        tol_res.append(test_class(decision,temp))
    accu=[x==y for x,y in zip(tol_res,y_true)].count(True)/len(test)
    return accu
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
data_wu=pd.read_csv('Trainning35070.csv')
test_wu=pd.read_csv('Testing35030.csv')
data_wu=prepro(data_wu)
test_wu=prepro(test_wu)
dec=decision_tree(data_wu,20)
accuracy(test_wu,dec) 